In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare what's needed for eORCA1 on the ice-shelf side

@author: Clara Burgard
"""

In [1]:
import xarray as xr
import numpy as np
from cdo import Cdo
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from datetime import date
#import xesmf as xe

In [2]:
%matplotlib qt5

In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 1.9.9rc1


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [5]:
file_Justine = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc').drop('quantile')

In [6]:
fNEMO_mask = 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'

REMAP THE ICE SHELF CONCENTRATION

In [7]:
input_grid = xr.open_dataset(inputpath_interim + 'IMBIE_2km_stereo_withbnds.nc')
# will be provided, was created with create_cdo_stereo_grid.ipynb

Prepare the output NEMO grid for conservative remapping

In [ ]:
# Load your curvilinear grid file
ds = xr.open_dataset(inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/coordinates.nc')

# Assuming the dataset has 'lat' and 'lon' variables
lat = ds['gphit']  # 2D array of latitudes
lon = ds['glamt'] # 2D array of longitudes
# Create a longitude with 0-360 values to avoir a gap in the bounds later
lon360 = lon.where(lon>0,lon+360)

# Create the lat and lon of the grid corners : ll = lower left, lr = lower right, ur = upper right, ul = upper left
meshlon_ll,meshlat_ll = (lon + lon.roll(x=-1))/2, (lat + lat.roll(x=-1))/2
meshlon_lr,meshlat_lr = (lon + lon.roll(x=1))/2, (lat + lat.roll(x=-1))/2
meshlon_ur,meshlat_ur = (lon + lon.roll(x=1))/2, (lat + lat.roll(x=1))/2
meshlon_ul,meshlat_ul = (lon + lon.roll(x=-1))/2, (lat + lat.roll(x=1))/2

# Create the lon of the grid corners using 0-360 : ll = lower left, lr = lower right, ur = upper right, ul = upper left
meshlon360_ll,_ = (lon360 + lon360.roll(x=-1))/2, (lat + lat.roll(x=-1))/2
meshlon360_lr,_ = (lon360 + lon360.roll(x=1))/2, (lat + lat.roll(x=-1))/2
meshlon360_ur,_ = (lon360 + lon360.roll(x=1))/2, (lat + lat.roll(x=1))/2
meshlon360_ul,_ = (lon360 + lon360.roll(x=-1))/2, (lat + lat.roll(x=1))/2

lon_bnds = xr.DataArray(data=np.array([meshlon_ll, meshlon_lr, meshlon_ur, meshlon_ul]), dims=['nvertex','y','x'])
lat_bnds = xr.DataArray(data=np.array([meshlat_ll, meshlat_lr, meshlat_ur, meshlat_ul]), dims=['nvertex','y','x'])
lon360_bnds = xr.DataArray(data=np.array([meshlon360_ll, meshlon360_lr, meshlon360_ur, meshlon360_ul]), dims=['nvertex','y','x']) 

# region where longitude hops from 180 to -180
prob_region = (lon <= -140) | (lon >= 140) & (np.abs(lon_bnds) < 25)
new_lon_bnds = lon_bnds.where(~prob_region, lon360_bnds)
new_lon_bnds = new_lon_bnds.where(new_lon_bnds <= 180, new_lon_bnds - 360)

# format the dataset
NEMO_new_grid = xr.merge([new_lon_bnds.rename('lon_bnds'),
                          lat_bnds.rename('lat_bnds')]).assign_coords({'lon': lon, 'lat': lat})

NEMO_new_grid['lon'].attrs['standard_name'] = 'longitude'
NEMO_new_grid['lon'].attrs['units'] = 'degrees_east'
NEMO_new_grid['lon'].attrs['bounds'] = 'lon_bnds'

NEMO_new_grid['lat'].attrs['standard_name'] = 'latitude'
NEMO_new_grid['lat'].attrs['units'] = 'degrees_north'
NEMO_new_grid['lat'].attrs['bounds'] = 'lat_bnds'

NEMO_new_grid['dummy'] = xr.DataArray(data=np.zeros(lon.shape), dims=['y','x'])

output_grid = NEMO_new_grid[['dummy','lon','lat','lon_bnds','lat_bnds']].set_coords(
    ("lat_bnds", "lon_bnds")).transpose('y','x','nvertex')
output_grid.to_netcdf(inputpath_interim + 'NEMO_grid_withbnds.nc')

Conservative regridding of the ice-shelf concentration

In [8]:
output_grid = xr.open_dataset(inputpath_interim + 'NEMO_grid_withbnds.nc')

In [ ]:
output_grid

In [ ]:
regridder_con = xe.Regridder(input_grid, output_grid, "conservative", periodic=True, ignore_degenerate=True)

In [ ]:
remapped_isf_conc = regridder_con(file_Justine['isf_conc'])

In [ ]:
remapped_isf_conc.to_dataset(name='isf_conc').to_netcdf(inputpath_interim + 'try_conservative_regridding_isf_conc.nc')

In [ ]:
# cdo remapcon,NEMO_grid_withbnds.nc -selvar,isf_conc Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc isfconc_NEMOgrid.nc
#cdo    remapcon (Abort): Source grid cell corner coordinates missing!

Double-check if what is represented in NEMO is point with 100% ice-shelf concentration 

In [ ]:
mask_1deg = (ds_isfNEMO['mask_isf'] > 0)

In [ ]:
ds_out.where(mask_1deg)

REMAP THE MASKS WITH NEAREST NEIGHBOUR

In [ ]:
#regridder_nn = xe.Regridder(input_grid, output_grid, "nearest_s2d", periodic=True) # this takes FOREVER!!!
# isfmask = regridder_nn(file_Justine['Iceshelf_extrap'])

In [11]:
isfmask_nn = cdo.remapnn(inputpath_interim + 'NEMO_grid_withbnds.nc',input=file_Justine['Iceshelf_extrap'], returnArray='Iceshelf_extrap')
isfmask = xr.DataArray(data=isfmask_nn, dims=['y','x'])

!!!!!!! ONLY IF YOUR GRID IS eORCA1 !!!!!!! => I think this is not needed anymore

In [ ]:
# For eORCA1, this method leads to no ocean cells for Bach (ID=101) and Cosgrove (ID=88). 
# I suggest merging it with its neighbour 'Britten' (ID=100), and neighbour 'No Name' (ID=89) 
# which has no melt in most of the obs products
#isfmask = isfmask.where(isfmask != 100, 101)
#isfmask = isfmask.where(isfmask != 89, 88)

CREATE MASK_VARIABLE

In [12]:
domain_cfg = xr.open_dataset(inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')


In [13]:
ds_isfNEMO = xr.Dataset()

ds_isfNEMO['mask_isf'] = isfmask.where(domain_cfg['isf_draft'] > 0)
ds_isfNEMO['mask_isf'].attrs['standard_name'] = 'mask of all the ice shelves'
ds_isfNEMO['mask_isf'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

#ds_isfNEMO['mask_isf_onlyisfconc1'] = ds_isfNEMO['mask_isf'].where(isfconc['isf_conc'] > 0.9999)


In [14]:
ds_isfNEMO['floating_frac'] = remapped_isf_conc # .where(isfconc['isf_conc'] < 0.9999,1)

NameError: name 'remapped_isf_conc' is not defined

Investigate stuff about isf conc

In [15]:
remapped_isf_conc = xr.open_dataset(inputpath_interim + 'try_conservative_regridding_isf_conc.nc')['isf_conc']

In [16]:
ds_isfNEMO = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc')

In [ ]:
((remapped_isf_conc > 0).astype(int) + (np.isfinite(ds_isfNEMO['mask_isf'])).astype(int)).plot()

In [ ]:
ds_isfNEMO['mask_isf'].plot()

CREATE MASK_ISF_OPEN AND MASK_ISF_CLOSED

In [ ]:
## gives the list of names when needed to better understand stuff
#for iid in file_Justine.ID:
#    print(iid.values,file_Justine['NAME'].sel(ID=iid).values)

In [17]:
# Choose (by eye or given a criterion, which ice shelves will have open cavities)
ID_open_list = [21,66,67,117,124,125,127,128]

In [18]:
# Divide the ref file into "open" and "closed" cavities
file_Justine_open = file_Justine.sel(ID=ID_open_list)
file_Justine_closed = file_Justine.drop_sel(ID=ID_open_list)

In [19]:
# Prepare domain_cfg closing (setting to 0) all information about the closed cavities
domain_cfg_closed =  domain_cfg.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry', 'mask_opensea', 'mask_csgrpglo']:
        domain_cfg_closed[vv] = domain_cfg_closed[vv].where(ds_isfNEMO['mask_isf'] != id_closed.values, 0)

  0%|          | 0/125 [00:00<?, ?it/s]

In [20]:
# Prepare mask of only open cavities 
mask_isf_open = ds_isfNEMO['mask_isf'].copy()
for id_closed in tqdm(file_Justine_closed.ID):
    mask_isf_open = mask_isf_open.where(ds_isfNEMO['mask_isf'] != id_closed.values)

  0%|          | 0/125 [00:00<?, ?it/s]

In [21]:
# Prepare mask of only closed cavities 
mask_isf_closed = ds_isfNEMO['mask_isf'].where(np.isnan(mask_isf_open))

In [22]:
# create mask_isf_open and mask_isf_closed
ds_isfNEMO['mask_isf_open'] = mask_isf_open
ds_isfNEMO['mask_isf_open'].attrs['standard_name'] = 'mask of the ice shelves that are open'
ds_isfNEMO['mask_isf_open'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

ds_isfNEMO['mask_isf_closed'] = mask_isf_closed
ds_isfNEMO['mask_isf_closed'].attrs['standard_name'] = 'mask of the ice shelves that are closed'
ds_isfNEMO['mask_isf_closed'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

IDENTIFY FRONT

In [23]:
# Create a mask discriminating between land, ocean and ice shelf
mask_0_1_2 = ds_isfNEMO['mask_isf'] * 2
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(domain_cfg['isf_draft'] == 0, 200) # ice shelf

In [24]:
# Create a mask of the extrapolated domains and only keep the domains corresponding to the closed cavities
extrap_domains_closed = isfmask.copy()

for kisf in ID_open_list:
    mask_0_1_2_closed = mask_0_1_2.where(ds_isfNEMO['mask_isf'] != kisf, 400)
    extrap_domains_closed = extrap_domains_closed.where(extrap_domains_closed != kisf, np.nan)

In [25]:
# set all ice shelves to 300
mask_front0 = mask_0_1_2.where((mask_0_1_2 == 0) | (mask_0_1_2 == 400), 300).copy()

In [26]:
mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front0.shift(x=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(x=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,extrap_domains_closed)

For the ice shelves not resolved at all in eORCA1

In [27]:
# define the domains that have not an associated front yet
extrap_domains_double_closed = extrap_domains_closed.copy()

for kisf in file_Justine_closed.ID:
    if (mask_front == kisf).astype(int).sum() > 0:
        #print(kisf.values)
        extrap_domains_double_closed = extrap_domains_double_closed.where(extrap_domains_double_closed != kisf, np.nan).drop('ID')

In [ ]:
# To double check where the front points are
#iid = 89
#plt.figure()
#((mask_0_2) + (isfmask['Iceshelf_extrap'] == iid)).plot()
#plt.title(str(iid))

In [ ]:
# set all ice shelves to land to have a delimitation of the whole contour of Antarctica
mask_0_2 = mask_0_1_2.where(mask_0_1_2 != 200, 2)
mask_0_2 = mask_0_2.where(mask_0_2 != 400, 2)

mask_front_new = mask_0_2.copy()

# check all directions and set points at border between ocean and land(300-0) to 500
mask_front_new = mask_front_new.where((mask_0_2.shift(x=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(x=1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=1)-mask_0_2) <= 0,5)
# cut out all front points south of the y = 90 latitude and in the ocean domain
mask_front_new = mask_front_new.where((mask_front_new == 5) & (mask_front_new.y < 90) & (mask_0_2 == 0))
# set the ice shelf number of the extrapolated domain
mask_front_new = mask_front_new.where(mask_front_new != 5,extrap_domains_double_closed)

In [ ]:
# check that it is the right delimitation (i.e. on the ocean points)
((mask_front_new > 0) + mask_0_2).plot()

In [ ]:
# combining the masks of the fronts (mask_front and mask_front_new)
mask_front_all = mask_front.copy()
for iid in file_Justine_closed.ID:
    if (ds_isfNEMO['mask_isf_closed'] == iid).astype(int).sum().values == 0: # if the ice shelf is not resolved in your grid, add the mask front inferred for them
        #print(iid.values, file_Justine_closed['NAME'].sel(ID=iid).values, (mask_front_new == iid).astype(int).sum().values)
        mask_front_all = mask_front_all.combine_first(mask_front_new.where(mask_front_new == iid))

In [ ]:
mask_front_file = xr.Dataset()
mask_front_file['mask_front'] = mask_front_all
mask_front_file['ID'] = file_Justine['ID']
mask_front_file.to_netcdf(inputpath_interim + 'mask_fronts_eORCA1.nc')

In [ ]:
mask_front_file['mask_front'].plot()

MAKE MAPS OF ZMIN AND ZMAX ON THE FRONT

In [ ]:
zmin_front = mask_front_all.copy()
zmax_front = mask_front_all.copy()

for id_closed in tqdm(file_Justine_closed.ID):
    zmin_front = zmin_front.where(mask_front_all != id_closed, file_Justine_closed['z_perc01'].sel(ID=id_closed)).drop('ID')
    zmax_front = zmax_front.where(mask_front_all != id_closed, file_Justine_closed['z_perc99'].sel(ID=id_closed)).drop('ID')

check if the extremes or the percentiles are more "realistic" => personally I think we should take the 1st and 99th percentiles

In [ ]:
file_Justine_closed['z_min'].plot()
file_Justine_closed['z_perc01'].plot()

In [ ]:
file_Justine_closed['z_max'].plot()
file_Justine_closed['z_perc99'].plot()

DISTRIBUTE MELT AT THE FRONT NORMALISED BY CELL SIZE

In [ ]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']
melt_src = 'Davison' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
melt_flux = file_Justine_closed['Melt'+melt_src]

In [ ]:
melt_front = mask_front_all.copy()
for id_closed in tqdm(file_Justine_closed.ID):        
    cell_area_isf = cell_area.where(mask_front_all == id_closed)
    cell_area_isf_sum = cell_area.where(mask_front_all == id_closed).sum()
    
    if id_closed == 100:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=101) + melt_flux.sel(ID=100)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 88:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=89) + melt_flux.sel(ID=88)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 14:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=14) + melt_flux.sel(ID=15)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 20:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=20) + melt_flux.sel(ID=17) + melt_flux.sel(ID=18)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 102:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=102) + melt_flux.sel(ID=105) + melt_flux.sel(ID=107)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    else:
        melt_front = melt_front.where(mask_front_all != id_closed, melt_flux.sel(ID=id_closed) * cell_area_isf / cell_area_isf_sum).drop('ID')


Double-check that the total melt is matched

In [ ]:
melt_front_tot_list = []
for id_closed in tqdm(file_Justine_closed.ID):
    melt_front_tot_list.append(melt_front.where(mask_front_all == id_closed).sum().assign_coords({'ID': id_closed}))
melt_front_tot_xr = xr.concat(melt_front_tot_list, dim='ID')

In [ ]:
melt_front.sum()

In [ ]:
melt_front_tot_xr.sum()

In [ ]:
file_Justine_closed['Melt'+melt_src].sum()

In [ ]:
(file_Justine_closed['Melt'+melt_src] - melt_front_tot_xr).where(abs(file_Justine_closed['Melt'+melt_src] - melt_front_tot_xr) > 0.5, drop=True)

Check the missing ones

In [ ]:
((extrap_domains_closed == 96) + (mask_0_2 == 2)).astype(int).plot()

In [ ]:
file_Justine_closed['NAME'].sel(ID=96).load()

CLEAN AND REMOVE ALL FRONTS WHERE MELT IS ZERO

In [ ]:
zmin_front = zmin_front.where(melt_front > 0)
zmax_front = zmax_front.where(melt_front > 0)
melt_front = melt_front.where(melt_front > 0)

WRITE TO NETCDF

In [ ]:
ds_isfNEMO['z_min'] = zmin_front
ds_isfNEMO['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_min'].attrs['units'] = 'm below sea level'

ds_isfNEMO['z_max'] = zmax_front
ds_isfNEMO['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_max'].attrs['units'] = 'm below sea level'

ds_isfNEMO['melt_isf_closed'] = melt_front
ds_isfNEMO['melt_isf_closed'].attrs['standard_name'] = 'melt flux from '+melt_src+' distributed at the ice-shelf front'
ds_isfNEMO['melt_isf_closed'].attrs['units'] = 'Gt/yr'

In [ ]:
ds_isfNEMO.attrs=dict(Source='Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc), melt from '+melt_src,
                      Compatibility='The masked variables are compatible with '+fNEMO_mask,
                      Creator='C. Burgard and P. Mathiot ('+date.today().strftime("%b-%d-%Y")+')')

In [ ]:
ds_isfNEMO.to_netcdf(inputpath_interim + 'masks_for_eORCA1.nc') 

In [ ]:
# remark: the Davison melt is the average over the varying conditions (not the steady state)

In [ ]:
check_file = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc')

In [ ]:
check_file['mask_isf_open'].plot()